In [130]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 
import warnings 
warnings.filterwarnings('ignore')

# Models
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

# Ensemble models
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor

# Regularization models
from sklearn.linear_model import Ridge, Lasso

# Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# Model Evaluation 
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [131]:
df= pd.read_csv(r"C:\Users\Arvind Vasa\AI-ML\Projects\notebook\data\stud.csv")
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [132]:
df['total_score']= df['math_score'] + df['reading_score']+ df['writing_score']
df['average']= df['total_score']/3

In [133]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333


In [134]:
# df2= df.drop(['math_score','writing_score','reading_score','total_score'], axis=1)
x= df.drop(['average'], axis=1)
y= df['average']

In [135]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


In [136]:
cols= x.columns
cols

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course', 'math_score', 'reading_score',
       'writing_score', 'total_score'],
      dtype='object')

In [137]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


In [138]:
num_features= x.select_dtypes(exclude= 'object').columns
cat_features= x.select_dtypes(include= 'object').columns

In [139]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

scaler= StandardScaler()
oh_encoder= OneHotEncoder()
label_encoder= LabelEncoder()

ct= ColumnTransformer(
    [
        ("OneHotEncoder", oh_encoder, cat_features),
        ("StandardScaler", scaler, num_features)
    ]
)

In [140]:
x.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,total_score
0,female,group B,bachelor's degree,standard,none,72,72,74,218
1,female,group C,some college,standard,completed,69,90,88,247
2,female,group B,master's degree,standard,none,90,95,93,278
3,male,group A,associate's degree,free/reduced,none,47,57,44,148
4,male,group C,some college,standard,none,76,78,75,229


In [141]:
x_transformed = ct.fit_transform(x)

In [142]:
x_transformed.shape

(1000, 21)

In [143]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test= train_test_split(x_transformed,y, test_size=0.2, random_state=101)
x_train.shape

(800, 21)

In [144]:
def evaluate_model(actual, predicted):
    mae= mean_absolute_error(actual, predicted)
    mse= mean_squared_error(actual, predicted)
    rmse= np.sqrt(mse)
    r2_square= r2_score(actual, predicted)
    return mae, rmse, r2_square

In [145]:
models= {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor" :KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_performance= {}
r2_list= []

for model_name, model in models.items():
    # training
    model.fit(x_train, y_train)

    # prediction
    y_train_pred= model.predict(x_train)
    y_test_pred= model.predict(x_test)

    # Evaluate model
    mae_train, mse_train, r2_square_train= evaluate_model(y_train, y_train_pred)
    mae_test, mse_test, r2_square_test= evaluate_model(y_test, y_test_pred)

    model_performance[model_name]= {
        'mae_train': mae_train,
        'mse_train': mse_train,
        'r2_score_train': r2_square_train * 100,
        'mae_test': mae_test,
        'mse_test': mse_test,
        'r2_score_test': r2_square_test * 100
    }

In [146]:
for model, performance in model_performance.items():
    print(model, performance['r2_score_test'])

LinearRegression 100.0
Lasso 99.51913610222445
Ridge 99.99998233250757
K-Neighbors Regressor 97.3341053609918
Decision Tree 99.99103906791507
Random Forest Regressor 99.99343498990471
XGBRegressor 99.93517773098299
CatBoosting Regressor 99.95948465450505
AdaBoost Regressor 99.37454315862239
